Estimator = 협업 기능 도움, 분업해서 팀업할 수 있게 해준다.
---bagging

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense
import pandas as pd
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
df = pd.read_csv('credit_data.csv')

In [3]:
x_data = np.array(df.iloc[:497, 0:6], dtype=np.float32)
y_data = np.array(df.iloc[:497, 6], dtype=np.float32).reshape(-1, 1)
x_pred = np.array(df.iloc[497:, 0:6], dtype=np.float32)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [9]:
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))\
            .repeat()\
            .batch(12)\
            .make_one_shot_iterator()\
            .get_next()
    return {'x': dataset[0]}, dataset[1] #x는 dictionary로, y값은 label로 보내겠다


def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))\
            .repeat()\
            .batch(x_test.shape[0])\
            .make_one_shot_iterator()\
            .get_next()
    return {'x': dataset[0]}, dataset[1] #x는 dictionary로, y값은 label로 보내겠다

In [10]:
def model_fn(features, labels, mode): #얘는 기본적으로 이러한 변수를 넣어줘야 한다.!
                                #mode = train : 학습, Eval: 평가, Predict : 예측
#    TRAIN = mode == tf.estimator.ModeKeys.TRAIN---책!
    nInput = 6      # input layer의 neuron 개수
    nHidden = 12    # hidden layer의 neuron 개수
    nOutput = 1     # output layer의 neuron 개수

# hidden layer의 Weight (Wh)와 Bias (Bh)
    H1 = Dense(nHidden, activation='relu')(features['x']) #features에서 바로 넣어준다!
    predY = Dense(nOutput, activation='sigmoid')(H1)

# 학습
    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(binary_crossentropy(labels, predY))
        optimizer = tf.train.AdamOptimizer(0.01)
        global_step= tf.train.get_global_step()
        train = optimizer.minimize(loss, global_step)
    
        return tf.estimator.EstimatorSpec(mode=mode, train_op=train, loss=loss)
# 평가
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss= tf.reduce_mean(binary_crossentropy(labels, predY))
        accuracy = tf.metrics.accuracy(labels, tf.round(predY))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})

#예측
    elif mode == tf.estimator.ModeKeys.PREDICT:
        yClass = tf.round(predY)
        return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = {'prob' : predY , 'class' : yClass})

In [13]:
estimator= tf.estimator.Estimator(model_fn=model_fn)#model_fn으로 만든 함수 넣기

estimator.train(train_input_fn, steps=100) #data를 함수로 묶어 넣어줌

estimator.evaluate(test_input_fn, steps=1)

pred_input_fn = tf.estimator.inputs.numpy_input_fn({'x' : x_pred}, shuffle=False)

for y in estimator.predict(pred_input_fn):
    print('prob = ', y['prob'][0])
    print('class = ', y['class'][0])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\student\\AppData\\Local\\Temp\\tmp9myiczlm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000016266BA8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Chec